## This notebook is meant for accademic purposes only

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Indications : 

Lire les données qui se trouvent dans Train.csv dans un DataFrame

Transformer les données catégorielles (sous forme de chaîne de caractères) avec OrdinalEncoder

Remplir les valeurs vides avec la fonctionnalité `fillna` de pandas : `df = df.fillna(0)`

Faire un PCA pour trouver le bon nombre de clusters : 
- si coude : s'arrêter au nombre de clusters au niveau du coude
- sinon, s'arrêter au nb de clusters au niveau de 90% de variance cumulée.

Faire un KMeans. Tester avec le nombre de clusters déduits de PCA. tester également avec de nombre de clusters différents. Evaleur avec le score de silhouette (ce score est bon s'il est grand), faire un choix de nb de clusters et justifier. Ajouter une visualisation des clusters avec TSNE, commenter le résultat.

Faire un DBSCAN. Afficher les clusters obtenus avec TSNE. Attention, si on a trop de noise (-1) le clustering n'est pas bon! Il faut alors modifier les deux paramètres dont nous avons parlé en cours (distance et nombre minimum de voisins). 

J'attends que le code soit commenté avec des commentaires pertinents, dans lesquels vous expliquez les choix que vous avez faits. 

PS: Ne mettez pas avec des commentaires copiés d'ailleurs, car si c'est le cas, on peut se rendre compte facilement que ce n'est pas vous qui les avez écrits. Je préfère que vous fassiez des commentaires avec vos propres mots.

Bonne chance!

**Lecture de données et création du dataframe**

In [ ]:
df= pd.read_csv("/kaggle/input/classifying-customers-into-segments/Train.csv")
df.info()
df.head()

**Supprimons les colonnes non pertinentes**

In [ ]:
df.drop(columns=["ID", "Segmentation"], inplace=True)

**Transformation des données avec OrdinalEncoder**

Transformation des données catégorielles en utilisant OrdinalEncoder pour les convertir en valeurs numériques.

In [ ]:
oe = OrdinalEncoder()
oe.fit(df.loc[:,["Gender", "Ever_Married", "Graduated", "Profession", "Spending_Score", "Var_1"]])
np_transformed = oe.transform(df.loc[:,["Gender", "Ever_Married", "Graduated", "Profession", "Spending_Score", "Var_1"]])
df.loc[:,["Gender", "Ever_Married", "Graduated", "Profession", "Spending_Score", "Var_1"]] = np_transformed

**Remplir les valeurs manquante avec zéro**

Remplissage des valeurs manquantes avec des zéros à l'aide de la fonction fillna de Pandas. On supprime les valeurs double avec la fonction drop_duplicates.

In [ ]:
df = df.fillna(0)
df=df.drop_duplicates()

In [ ]:
df.head()

**Déterminons le nombre de cluster à partir du PCA**

Réalisons le PCA pour déterminer le bon nombre de clusters. On retiendras 3 composants pour réaliser pour la suite de l'analyse.

In [ ]:
# Réaliser une PCA pour trouver le bon nombre de clusters
pca = PCA()
pca.fit(df)

ratio = pca.explained_variance_ratio_
print(ratio)
cum_ratio = np.cumsum(ratio)
print(cum_ratio)
x = list(range(1, len(ratio)+1))
x_legend = ["p"+str(i) for i in x]
plt.bar(x_legend, ratio)
plt.plot(x_legend, cum_ratio, 'r-')

plt.show()

On déduit 3 clusters à partir du résultat du PCA

**K-means et Score silhouette pour déterminer le nombre optimal de cluster**

In [ ]:
NB_CLUSTERS=3
data = np.zeros((NB_CLUSTERS-1,2))
for j in range(2,NB_CLUSTERS+1):
    km = KMeans(n_clusters=j)
    km.fit(X_pca)
    lab = km.labels_
    score = silhouette_score(X_pca,lab)
    data[j-2,:] = (j,score)
    print("Nombre clusters : ", j, " Le score de silhouette est : ", score)

Le score silhouette le plus grand est celui correspondant à 2 clusters. Donc le nombre optimal de cluster est 2

**Visualisation des clusters avec le TSNE**

Utilisons la PCA pour réduire les dimensions des données à 3 dimensions.

In [ ]:
pca = PCA(n_components=3)
pca.fit(df)
X_pca = pca.transform(df)
kmeans = KMeans(n_clusters=2)
kmeans.fit(X_pca)
y_pred = kmeans.predict(X_pca)

# Réduire les dimensions des features
X_tsne = tsne.fit_transform(X_pca)

# Afficher le scatter plot 2D

tsne = TSNE(n_components=2, perplexity=30)
X_tsne = tsne.fit_transform(X_pca)
plt.scatter(X_tsne[:,0], X_tsne[:,1], c=y_pred)
plt.show()

Le TSNE nous affiche les deux cluster trouvés grace au K-means.

**DBSCAN**

In [ ]:
# Instancier l'objet t-SNE
tsne = TSNE(n_components=2, perplexity=30, random_state=0)

# Réduire les dimensions des features avec PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(df)

# DBSCAN
dbscan = DBSCAN(eps=2.5, min_samples=10)
y_pred = dbscan.fit_predict(X_pca)

# Réduire les dimensions avec t-SNE
X_tsne = tsne.fit_transform(X_pca)

# Afficher le scatter plot 2D avec les clusters de DBSCAN
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y_pred)
plt.title('t-SNE visualisation avec les clusters de DBSCAN')
plt.xlabel('t-SNE feature 1')
plt.ylabel('t-SNE feature 2')
plt.colorbar()
plt.show()


Ce code permet de visualiser les clusters obtenus à partir d'une analyse de clustering en utilisant l'algorithme DBSCAN avec une réduction de dimension par PCA et t-SNE.